# DOCUMENTOS con Gemini

## Importaciones

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import io
import pathlib, base64, httpx

## Clave de la API

In [2]:
# Cargamos las variables de entorno desde .env e inicialiÇzamos la constante API_KEY
load_dotenv(override=True)
API_KEY = os.getenv('GEMINI_API_KEY') #si se llama así, no hace falta introducirla en el constructor de OpenAI

if not API_KEY:
    print("ERROR: No se ha encontrado la clave de la API")
else:
    print("Se ha encontrado la clave de la API correctamente")

Se ha encontrado la clave de la API correctamente


## Iniciación

In [3]:
MODEL_NAME = "gemini-1.5-flash"
genai.configure(api_key=API_KEY) 

In [4]:
system_prompt = "Utiliza un tono amable"
#system_prompt = "Utiliza un tono enfadado"

In [5]:
model = genai.GenerativeModel(
    model_name = MODEL_NAME,
    system_instruction = system_prompt,
    #generation_config = genai.GenerationConfig(
    #    max_output_tokens = 1000,
    #    temperature = 0.1,
    #),
)

## Un documento en local

In [6]:
doc = pathlib.Path('recursos/paper.pdf').read_bytes()

In [7]:
user_prompt = "Por favor, explícame que hay en el documento. Trata de detallar tanto como puedas. Además ponle un título"

In [8]:
response = model.generate_content([{'mime_type':'application/pdf', 'data': doc}, user_prompt])

In [9]:
print(response.text)

Claro que sí. Aquí tienes un resumen detallado del documento que has proporcionado, incluyendo un título sugerido:

**Título sugerido:** Guía para la elaboración de artículos científicos de investigación.

**Resumen:**

Este documento es una guía paso a paso para escribir un artículo científico de investigación, dirigido especialmente a principiantes en la escritura científica. Los autores, Elena Henríquez Fierro y María Inés Zepeda González, explican cada etapa del proceso de redacción, desde el título hasta las referencias bibliográficas.  El artículo está escrito en español e incluye un resumen y un abstract en inglés.

**Puntos clave que se tratan en el documento:**

* **Título:**  Se recomiendan las mejores prácticas para crear un título conciso, informativo y preciso, que no exceda las 15 palabras y que incluya las palabras clave del estudio.

* **Autores:** Se explica cómo presentar adecuadamente la información sobre los autores, incluyendo sus afiliaciones y correo electrónico 

## Un PDF en remoto

In [10]:
doc_path = "https://www.turismoasturias.es/documents/39908/3871406/Casonas%20Asturianas.pdf"
doc = base64.b64encode(httpx.get(doc_path).content).decode("utf-8")

In [11]:
user_prompt = "Por favor, resume el documento"

In [12]:
response = model.generate_content([{'mime_type':'application/pdf', 'data': doc}, user_prompt])

In [13]:
print(response.text)

Claro. Este es un resumen del folleto de Casonas Asturianas:

El folleto es una guía de los hoteles rurales de Asturias. Hay una introducción a la región y luego una lista de hoteles, clasificados por región (occidente, centro y oriente). Cada hotel tiene una breve descripción, incluyendo el número de estrellas, las instalaciones, el entorno y la información de contacto. También incluye fotos de cada hotel. Hay mapas que muestran la ubicación de cada hotel.

¿Hay algo más en lo que pueda ayudarte?


## Un PDF en la API de File

In [14]:
doc_path = "https://www.turismoasturias.es/documents/39908/3871406/Casonas%20Asturianas.pdf"
doc = io.BytesIO(httpx.get(doc_path).content) #la API espera el archivo como entrada e BytesIO da una representación del archivo en memoria en lugar de en disco
doc_file = genai.upload_file(path=doc, mime_type='application/pdf')

In [15]:
user_prompt = "Por favor, resume el documento"

In [16]:
response = model.generate_content(
    [doc_file, user_prompt], #pueden ponerse varios documentos
)

In [17]:
print(response.text)

Claro que sí. El documento es un catálogo para el Club de Calidad Casonas Asturianas. Describe los alojamientos del club, ofreciendo fotografías y explicaciones de sus comodidades y ubicación. También ofrece una breve descripción de las áreas del oeste y del este de Asturias donde se encuentran los hoteles, haciendo hincapié en los aspectos culturales y turísticos de las zonas.


## Ver archivos subidos a la plataforma

In [18]:
for f in genai.list_files():
    print("  ", f)

   genai.File({
    'name': 'files/og9qcck4nl2b',
    'display_name': '',
    'mime_type': 'application/pdf',
    'sha256_hash': 'Y2JmZTAxZTIyZWEzYmFhNTAzOGJkZjAyNWQzZDNjNzRiYzVhZmY2MTA0OGRlZTcwZmViMmUwZTgwMzJmMWE1Mg==',
    'size_bytes': '6596651',
    'state': 'ACTIVE',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/og9qcck4nl2b',
    'create_time': '2025-02-09T13:58:43.807878Z',
    'expiration_time': '2025-02-11T13:58:43.724667262Z',
    'update_time': '2025-02-09T13:58:43.807878Z'})


## Borrar archivos subidos

In [19]:
for f in genai.list_files():
    f.delete()

## Ver archivos subidos

In [20]:
for f in genai.list_files():
    print("  ", f.name)